In [1]:
from glob import glob
import os
import torch
import torchmetrics
import nibabel as nib
import subprocess
import numpy as np
import torchmetrics
import pandas as pd

In [2]:
region_names = ["bg", "hy", "mb", "hpf", "str", "ctx", "gp", "th", "cb", "olf", "bs"]

In [3]:
atlas_mask = "dataset3/Atlas/P56_Annotation_128_norm_id_mask.nii.gz"
atlas_labels = "dataset3/Atlas/GIN_LabelsFusion_MajorityVoting_affine.nii.gz"

affine_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'Labels', "*_affine_convert.nii.gz")))
deep1_labels = sorted(glob(os.path.join('output', 'GIN', 'scenario1', 'Labels', '*.nii.gz')))
deep2_labels = sorted(glob(os.path.join('output', 'GIN', 'scenario2', 'Labels', '*.nii.gz')))
deep3_labels = sorted(glob(os.path.join('output', 'GIN', 'scenario3', 'Labels', '*.nii.gz')))
ants_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'Labels', "*_deformable_convert.nii.gz")))
mean_labels = sorted(glob(os.path.join('dataset3', 'GIN_Mean', 'Labels', "*_deformable_convert.nii.gz")))

dicemetric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=11)

values_affine = [0,0,0,0,0,0,0,0,0,0,0]
values_deep1 = [0,0,0,0,0,0,0,0,0,0,0]
values_deep2 = [0,0,0,0,0,0,0,0,0,0,0]
values_deep3 = [0,0,0,0,0,0,0,0,0,0,0]
values_ants = [0,0,0,0,0,0,0,0,0,0,0]
values_mean = [0,0,0,0,0,0,0,0,0,0,0]

mapping_csv = "dataset3/GIN/labels_mapping.csv"
data = pd.read_csv(mapping_csv)
    
map6 = torch.from_numpy(nib.load(atlas_labels).get_fdata().ravel()).type('torch.LongTensor')
count = 0

list_dict = []

for i, _ in enumerate(ants_labels):  
    count += 1
    affine = torch.from_numpy(nib.load(affine_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    affine[np.where(affine > 10)] = 0
    deep1 = torch.from_numpy(nib.load(deep1_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    deep1[np.where(deep1 > 10)] = 0
    deep2 = torch.from_numpy(nib.load(deep2_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    deep2[np.where(deep2 > 10)] = 0
    deep3 = torch.from_numpy(nib.load(deep3_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    deep3[np.where(deep3 > 10)] = 0
    ants = torch.from_numpy(nib.load(ants_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    ants[np.where(ants > 10)] = 0  
    mean = torch.from_numpy(nib.load(mean_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    mean[np.where(mean > 10)] = 0  

    affine_values = dicemetric(affine,map6)
    deep1_values = dicemetric(deep1,map6)
    deep2_values = dicemetric(deep2,map6)
    deep3_values = dicemetric(deep3,map6)
    ants_values = dicemetric(ants,map6)    
    mean_values = dicemetric(mean,map6)  
    for j in range(len(deep1_values)):
        values_affine[j] = affine_values[j].item()
        values_deep1[j] = deep1_values[j].item()
        values_deep2[j] = deep2_values[j].item()
        values_deep3[j] = deep3_values[j].item()
        values_ants[j] = ants_values[j].item()
        values_mean[j] = mean_values[j].item()
    name = ants_labels[i].split('/')[-1].split('_')[0]    
    datatype = "MAP6" if "M" in name else "SVBP"    
    affine_dict = {"affine_dice_" + str(region_names[k]): values_affine[k] for k in range(len(region_names))}
    ants_dict = {"ants_dice_" + str(region_names[k]): values_ants[k] for k in range(len(region_names))}
    deep1_dict = {"deep1_dice_" + str(region_names[k]): values_deep1[k] for k in range(len(region_names))}
    deep2_dict = {"deep2_dice_" + str(region_names[k]): values_deep2[k] for k in range(len(region_names))}
    deep3_dict = {"deep3_dice_" + str(region_names[k]): values_deep3[k] for k in range(len(region_names))}
    mean_dict = {"antsmean_dice_" + str(region_names[k]): values_mean[k] for k in range(len(region_names))}
    ith_dict = {"name": name, "type": datatype}
    ith_dict.update(affine_dict)
    ith_dict.update(ants_dict)
    ith_dict.update(deep1_dict)
    ith_dict.update(deep2_dict)
    ith_dict.update(deep3_dict)
    ith_dict.update(mean_dict)
    list_dict.append(ith_dict)
print(ith_dict)
df = pd.DataFrame(list_dict)
df.to_csv("dice_majorityaffine_results.csv")


{'name': 'M8876', 'type': 'MAP6', 'affine_dice_bg': 0.9879671931266785, 'affine_dice_hy': 0.8749484419822693, 'affine_dice_mb': 0.788844645023346, 'affine_dice_hpf': 0.9268980026245117, 'affine_dice_str': 0.9074997305870056, 'affine_dice_ctx': 0.9358503222465515, 'affine_dice_gp': 0.7292057275772095, 'affine_dice_th': 0.9165790677070618, 'affine_dice_cb': 0.9164120554924011, 'affine_dice_olf': 0.8384580016136169, 'affine_dice_bs': 0.0007000758196227252, 'ants_dice_bg': 0.9844911694526672, 'ants_dice_hy': 0.8069013357162476, 'ants_dice_mb': 0.6978312134742737, 'ants_dice_hpf': 0.870882511138916, 'ants_dice_str': 0.8695815205574036, 'ants_dice_ctx': 0.8945319652557373, 'ants_dice_gp': 0.7004662156105042, 'ants_dice_th': 0.855911135673523, 'ants_dice_cb': 0.8781678080558777, 'ants_dice_olf': 0.6789211630821228, 'ants_dice_bs': 0.0002582144516054541, 'deep1_dice_bg': 0.9865360856056213, 'deep1_dice_hy': 0.8805505633354187, 'deep1_dice_mb': 0.7940173745155334, 'deep1_dice_hpf': 0.9225494861